<a href="https://colab.research.google.com/github/shafieiali42/Question-answering/blob/main/Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook author: Ali Shafiei

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!!pip install git+https://github.com/huggingface/accelerate
!apt install git-lfs
!git config --global user.email "shafieiali42@gmail.com"
!git config --global user.name "shafieiali42"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [2]:
import numpy as np
from tqdm import tqdm
from tqdm.auto import tqdm
import collections
import evaluate
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator
import torch
from transformers import AutoModelForQuestionAnswering

In [3]:
from datasets import load_dataset

# restaurants_dataset = load_dataset("subjqa", "restaurants")
restaurants_dataset = load_dataset("subjqa", "movies")

Generating train split:   0%|          | 0/1369 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/291 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/261 [00:00<?, ? examples/s]

Dataset subjqa downloaded and prepared to /root/.cache/huggingface/datasets/subjqa/movies/1.1.0/2c12e496c4c675ab4a57ffb5d3f538f2e7b89793956e50da37126393ce23b6c6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## Data Cleaning

In [4]:
restaurants_dataset

DatasetDict({
    train: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 1369
    })
    test: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 291
    })
    validation: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 261
    })
})

In [5]:
def count_multiple_answers(data):
  counter=0
  for i in range(len(data)):
    if len(data[i]["answers"]["text"])>1:
      print(data[i]["question"])
      print(data[i]["context"])
      print(data[i]["answers"]["text"])
      print(i)
      print("-"*500)
      # print(len(data[i]["answers"]["text"]))
      counter+=1
  return counter

print(f'{count_multiple_answers(restaurants_dataset["train"])}/{len(restaurants_dataset["train"])}')
# print(f'{count_multiple_answers(restaurants_dataset["validation"])}/{len(restaurants_dataset["validation"])}')
# print(f'{count_multiple_answers(restaurants_dataset["test"])}/{len(restaurants_dataset["test"])}')

# print(f'{count_multiple_answers(books_dataset["train"])}/{len(books_dataset["train"])}')
# print(f'{count_multiple_answers(books_dataset["validation"])}/{len(books_dataset["validation"])}')
# print(f'{count_multiple_answers(books_dataset["test"])}/{len(books_dataset["test"])}')


How is film?
"The Passion Of The Christ" is brilliant and one of the best pictures of the year, so far! The movie focuses on the last 12 hours of Jesus' (Jim Caviezel) life leading up to his death and resurrection. This movie has some very graphic scenes, but don't let the violence concern you. It is a powerful depiction of the last 12 hours of Jesus' life, thanks in part to its director, the brilliant Mel Gibson, who directed and starred as Justin Mcleod in "The Man Without A Face" and William Wallace in "Braveheart", the latter, which won him the Best Director Oscar. He also well known as Dr. Gibbon in "The Singing Detective" (2003), Graham Hess in "Signs" (2002), Lt. Col. Hal Moore in "We Were Soldiers" (2002), Nick Marshall in "What Women Want" (2000), Benjamin Martin in "The Patriot" (2000), Porter in "Payback" (1999), Martin Riggs in the Lethal Weapon movies (1987, 1989, 1992, 1998), Jerry Fletcher in "Conspiracy Theory" (1997), Tom Mullen in "Ransom" (1996), the voices of Rocky 

In [6]:
print("Context: ", restaurants_dataset["train"][0]["context"])
print("Question: ", restaurants_dataset["train"][0]["question"])
print("Answer: ", restaurants_dataset["train"][0]["answers"])

Context:  Well, I have to admit I wasn't expecting to like this movie, having read all the bad press surrounding it.  But I saw it last night on a whim and I have to say I thought it was fantastic!  Firstly, as a movie it is incredibly well shot.  The scenery is wonderful and the sets are painstakingly recreated - never once do you doubt this is 1st century Palestine.The acting starts off a little strained in the garden scene, when the movie is just finding it's feet (and the audience is coming to terms with the fact that the movie isn't some horrible, cheap propaganda short, but a proper movie) but quickly becomes impressive and ends up brutally real.Gibson has set out to make a compelling dramatization of the last 12 hours of Jesus' life as described in the Gospels and that's pretty much what he does.  There is certain artistic license taken and some good abstract movie-making, but nothing that really detracts from the movie or the sentiment.  In fact the scenes where they have the B

In [7]:
# model_checkpoint = "bert-base-cased"
model_checkpoint = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
def add_zero(example):
  if len( example["answers"]["answer_start"])==0:
    example["answers"]["answer_start"]=[-1000]
  if len( example["answers"]["text"])==0:
    example["answers"]["text"]=[""]
  if len( example["answers"]["answer_subj_level"])==0:
    example["answers"]["answer_subj_level"]=[0]
  if len( example["answers"]["ans_subj_score"])==0:
    example["answers"]["ans_subj_score"]=[0]
  if len( example["answers"]["is_ans_subjective"])==0:
    example["answers"]["is_ans_subjective"]=[False]
  return example

def remove_second_answers(example):
  if len(example["answers"]["answer_start"])>=2:
    example["answers"]["answer_start"]=[example["answers"]["answer_start"][0]]
  if len(example["answers"]["text"])>=2:
    example["answers"]["text"]=[example["answers"]["text"][0]]
  if len(example["answers"]["answer_subj_level"])>=2:
    example["answers"]["answer_subj_level"]=[example["answers"]["answer_subj_level"][0]]
  if len(example["answers"]["ans_subj_score"])>=2:
    example["answers"]["ans_subj_score"]=[example["answers"]["ans_subj_score"][0]]
  if len(example["answers"]["is_ans_subjective"])>=2:
    example["answers"]["is_ans_subjective"]=[example["answers"]["is_ans_subjective"][0]]
  return example

x=0
def is_same(str1,str2):
  global x
  matched=len(list(set(str1.split()) & set(str2.split())))
  # print(matched)
  matched_frac=matched/len(str1.split())
  if(matched_frac>0.6):
    x+=1
    print(str1,str2)
    print("-"*500)

  return matched_frac>0.6
def remove_duplicated_answers(example):
  indexes=[]
  for i in range(len(example["answers"]["text"])):
    for j in range(i+1,len(example["answers"]["text"])):
      if is_same(example["answers"]["text"][i],example["answers"]["text"][j]):
        print(example["answers"]["text"])
        indexes.append(j)
  a1=[]
  a2=[]
  a3=[]
  a4=[]
  a5=[]
  # print(example["answers"])
  for i in range(len(example["answers"]["text"])):
    if i not in indexes:
      a1.append(example["answers"]["text"][i])
      a2.append(example["answers"]["answer_start"][i])
      a3.append(example["answers"]["answer_subj_level"][i])
      a4.append(example["answers"]["ans_subj_score"][i])
      a5.append(example["answers"]["is_ans_subjective"][i])

  example["answers"]["text"]=a1
  example["answers"]["answer_start"]=a2
  example["answers"]["answer_subj_level"]=a3
  example["answers"]["ans_subj_score"]=a4
  example["answers"]["is_ans_subjective"]=a5
  return example

restaurants_dataset["train"]=restaurants_dataset["train"].map(add_zero)
restaurants_dataset["validation"]=restaurants_dataset["validation"].map(add_zero)
restaurants_dataset["test"]=restaurants_dataset["test"].map(add_zero)
restaurants_dataset["train"]=restaurants_dataset["train"].map(remove_duplicated_answers)
restaurants_dataset["train"]=restaurants_dataset["train"].map(remove_second_answers)
print(x)

Map:   0%|          | 0/1369 [00:00<?, ? examples/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/291 [00:00<?, ? examples/s]

Map:   0%|          | 0/1369 [00:00<?, ? examples/s]

it's great fun I don't  expect this film to become a classic, but it's great fun that can be  watched over and over again
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
["it's great fun", "I don't  expect this film to become a classic, but it's great fun that can be  watched over and over again"]
A very brutal and terrifyingly emotional piece A very brutal and terrifyingly emotional piece
----------------------------------------------------------------------------------------------------------------------------------------------------------

Map:   0%|          | 0/1369 [00:00<?, ? examples/s]

88


## Pre-processing

In [9]:
restaurants_dataset["train"][0]

{'domain': 'movies',
 'nn_mod': 'shocking',
 'nn_asp': 'movie',
 'query_mod': 'brutal',
 'query_asp': 'movie',
 'q_reviews_id': 'b9ccc5b31d7d8088e2c1ae7c9abd29d3',
 'question_subj_level': 1,
 'ques_subj_score': 0.0,
 'is_ques_subjective': False,
 'review_id': '08b48e385ef94c083315a2a48d6dfa3a',
 'id': '4593a383c6b34dc5b3862746cc792675',
 'title': '0310263662',
 'context': 'Well, I have to admit I wasn\'t expecting to like this movie, having read all the bad press surrounding it.  But I saw it last night on a whim and I have to say I thought it was fantastic!  Firstly, as a movie it is incredibly well shot.  The scenery is wonderful and the sets are painstakingly recreated - never once do you doubt this is 1st century Palestine.The acting starts off a little strained in the garden scene, when the movie is just finding it\'s feet (and the audience is coming to terms with the fact that the movie isn\'t some horrible, cheap propaganda short, but a proper movie) but quickly becomes impressi

In [10]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    # sample_map = inputs.pop("overflow_to_sample_mapping")
    sample_map = inputs["overflow_to_sample_mapping"]
    answers = examples["answers"]
    ids=examples["id"]
    start_positions = []
    end_positions = []
    ids_list=[]

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        question=questions[sample_idx]
        id=ids[sample_idx]
        ids_list.append(id)
        # print(answer)
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)


        if start_char<0:
          start_positions.append(0)
          end_positions.append(0)
          continue

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
        # if examples["question"][sample_idx]=="Who has the best ramen?":
        #     print(question)
        #     print(sequence_ids)
        #     print(examples["answers"][sample_idx]["answer_start"])
        #     print(examples["answers"][sample_idx]["text"])
        #     print(start_positions[-1])
        #     print(end_positions[-1])

    inputs["ids"]=ids_list
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    print(inputs.keys())
    return inputs

In [11]:
train_dataset = restaurants_dataset["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=restaurants_dataset["train"].column_names,
)
len(restaurants_dataset["train"]), len(train_dataset)

Map:   0%|          | 0/1369 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'ids', 'start_positions', 'end_positions'])
dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'ids', 'start_positions', 'end_positions'])


(1369, 2655)

In [12]:
from pandas.core.indexes.category import contains
def check_preprocessing(idx):
  # sample_idx = train_dataset[idx]["overflow_to_sample_mapping"]
  # answer = restaurants_dataset["train"][sample_idx]["answers"]["text"][0]
  answer=""
  for index_restaurant in range(len(restaurants_dataset["train"])):
    if restaurants_dataset["train"][index_restaurant]["id"]==train_dataset[idx]["ids"]:
      # print(restaurants_dataset["train"][index_restaurant]["question"])
      answer=restaurants_dataset["train"][index_restaurant]["answers"]["text"][0]

  start = train_dataset[idx]["start_positions"]
  end = train_dataset[idx]["end_positions"]
  labeled_answer = tokenizer.decode(train_dataset[idx]["input_ids"][start:end+1])
  # print(f"Theoretical answer: {answer}")
  # print(f"labels give: {labeled_answer}")
  # print(len(answer))
  # print(len(labeled_answer.strip()))
  # print(labeled_answer.__contains__(answer))
  if answer==labeled_answer:
    return True
  elif labeled_answer=="[CLS]" and answer=='':
    return True
  elif labeled_answer=="[CLS]" and not answer.__contains__(labeled_answer):
    return True
  return False

In [13]:
random_list=np.random.randint(low = 0,high=len(train_dataset),size=10)
# print(check_preprocessing(830))
for i in tqdm(random_list):
  if not check_preprocessing(int(i)):
    print(i)
# check_preprocessing(0)
# check_preprocessing(1)

  0%|          | 0/10 [00:00<?, ?it/s]

81
272
1956
1712
940
2338
1307
2284
1247
2278


In [14]:
print(train_dataset[0]["input_ids"])
# print(train_dataset[0]["token_type_ids"])
print(train_dataset[0]["attention_mask"])
print(train_dataset[0]["start_positions"])
print(train_dataset[0]["end_positions"])

[0, 6179, 222, 47, 101, 5, 1569, 116, 2, 2, 8346, 6, 38, 33, 7, 8109, 38, 938, 75, 4804, 7, 101, 42, 1569, 6, 519, 1166, 70, 5, 1099, 1228, 3817, 24, 4, 1437, 125, 38, 794, 24, 94, 363, 15, 10, 31754, 8, 38, 33, 7, 224, 38, 802, 24, 21, 5500, 328, 1437, 30800, 6, 25, 10, 1569, 24, 16, 5784, 157, 738, 4, 1437, 20, 26815, 16, 4613, 8, 5, 3880, 32, 31751, 352, 35244, 1070, 111, 393, 683, 109, 47, 2980, 42, 16, 112, 620, 3220, 16398, 4, 133, 3501, 2012, 160, 10, 410, 15718, 11, 5, 5671, 1310, 6, 77, 5, 1569, 16, 95, 2609, 24, 18, 1730, 36, 463, 5, 2437, 16, 567, 7, 1110, 19, 5, 754, 14, 5, 1569, 965, 75, 103, 11385, 6, 6162, 10934, 765, 6, 53, 10, 4692, 1569, 43, 53, 1335, 3374, 3444, 8, 3587, 62, 23134, 588, 4, 534, 1452, 1478, 34, 278, 66, 7, 146, 10, 9669, 26274, 415, 1938, 9, 5, 94, 316, 722, 9, 5772, 108, 301, 25, 1602, 11, 5, 272, 46542, 8, 14, 18, 1256, 203, 99, 37, 473, 4, 1437, 345, 16, 1402, 11419, 4385, 551, 8, 103, 205, 20372, 1569, 12, 5349, 6, 53, 1085, 14, 269, 38442, 29, 31

## Pre-process Validation set

In [15]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 or k==0 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [16]:
validation_dataset = restaurants_dataset["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=restaurants_dataset["validation"].column_names,
)
len(restaurants_dataset["validation"]), len(validation_dataset)

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

(261, 527)

In [17]:
validation_dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'example_id'])

In [18]:
print(validation_dataset[0]["input_ids"])
print(validation_dataset[0]["offset_mapping"])
print(validation_dataset[0]["example_id"])


[0, 6179, 21, 5, 1569, 116, 2, 2, 4148, 902, 564, 6, 4482, 23, 195, 35, 541, 2784, 6, 38, 938, 75, 2230, 686, 141, 7, 619, 142, 38, 21, 2828, 11, 760, 9, 10, 2441, 14, 21, 1227, 7, 695, 99, 18, 57, 373, 65, 9, 5, 144, 4456, 8, 8513, 3541, 655, 156, 4, 38, 21, 1227, 7, 6307, 11, 13, 103, 22365, 6, 53, 89, 58, 4146, 4, 20, 822, 554, 12810, 15, 3078, 6, 8, 38, 21, 416, 551, 36347, 30, 5, 7207, 23666, 9, 5, 822, 4, 38, 938, 75, 686, 141, 7, 3886, 2185, 4, 38, 1467, 14, 38, 21, 164, 7, 4562, 11783, 16420, 9, 822, 6, 53, 5, 898, 21, 55, 87, 38, 64, 224, 4, 17449, 342, 24, 6, 22, 133, 35477, 9, 5, 4845, 113, 16, 41, 676, 14, 1395, 28, 1602, 11, 1617, 6, 53, 129, 11, 5, 1808, 1026, 14, 4448, 9909, 34, 1412, 7, 5486, 10, 2157, 101, 117, 97, 4, 96, 42, 1551, 6, 38, 40, 31278, 59, 5, 822, 145, 5, 275, 655, 4, 2612, 116, 3047, 24, 16, 72, 133, 35477, 60, 25, 144, 486, 24, 11, 765, 6, 16, 45, 129, 4448, 9909, 18, 14535, 173, 36, 700, 56, 1433, 3660, 22, 46045, 11567, 113, 8, 22, 133, 1554, 6830, 10

## Eval on small set

In [19]:
def normalize_text(s):
    import string, re
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [20]:
def compute_f1_between_two_str(pred_Str,true_str):
  pred_tokens = normalize_text(pred_Str).split()
  truth_tokens = normalize_text(true_str).split()
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  common_tokens = set(pred_tokens) & set(truth_tokens)
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  return 2 * (prec * rec) / (prec + rec)



def compute_em_between_two_str(pred_Str,true_str):
  return int(normalize_text(pred_Str) == normalize_text(true_str))

def compute_f1_em(true_answers,predicted_answers):
  f1_list=[]
  em_list=[]
  for i in range(len(true_answers)):
    true=true_answers[i]
    all_trues=true["answers"]["text"]
    pred=predicted_answers[i]["prediction_text"]
    f1_scores=[]
    em_scores=[]
    for j in range(len(all_trues)):
      f1_scores.append(compute_f1_between_two_str(pred,all_trues[j]))
      em_scores.append(compute_em_between_two_str(pred,all_trues[j]))
    f1_list.append(max(f1_scores))
    em_list.append(max(em_scores))
  f1_list=np.array(f1_list)
  em_list=np.array(em_list)
  return {"F1": np.mean(f1_list)*100, "EM": np.mean(em_list)*100}


In [21]:
# small_eval_set = restaurants_dataset["validation"].select(range(100))
# trained_checkpoint = "distilbert-base-cased-distilled-squad"
# tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
# eval_set = small_eval_set.map(
#     preprocess_validation_examples,
#     batched=True,
#     remove_columns=restaurants_dataset["validation"].column_names,
# )

In [22]:
# import torch
# from transformers import AutoModelForQuestionAnswering

# eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
# eval_set_for_model.set_format("torch")
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
# trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

In [23]:
# with torch.no_grad():
#     outputs = trained_model(**batch)
# start_logits = outputs.start_logits.cpu().numpy()
# end_logits = outputs.end_logits.cpu().numpy()

In [24]:
n_best = 20
max_answer_length = 30
# metric = evaluate.load("squad")
def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (end_index < start_index or end_index - start_index + 1 > max_answer_length):
                        continue
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    # print(type(answer["logit_score"]))
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": {'text':ex["answers"]['text'],"answer_start":ex["answers"]["answer_start"]}} for ex in examples]
    # print(predicted_answers)
    # print("-"*500)
    # print(theoretical_answers)
    # for idd in range(len(predicted_answers)):
      # print(len(predicted_answers[idd]["prediction_text"])==0)
      # print(predicted_answers[idd]["prediction_text"])

    return compute_f1_em(theoretical_answers,predicted_answers)

In [25]:
# compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

## Training

In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [27]:
train_dataset_for_model=train_dataset.remove_columns(["overflow_to_sample_mapping","ids"])
train_dataset_for_model.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset_for_model,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [28]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [29]:
from torch.optim import AdamW
optimizer = torch.optim.Adam(model.parameters(), lr=2e-05,betas=(0.9,0.999),eps=1e-08)
# optimizer = AdamW(model.parameters(), lr=2e-5)

In [30]:
from accelerate import Accelerator

# accelerator = Accelerator(fp16=True)
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [31]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [32]:
from huggingface_hub import Repository, get_full_repo_name

# model_name = "test-subjqa"
model_name="Fine_tuned_roberta-base-squad2"
repo_name = get_full_repo_name(model_name)
repo_name

'shafieiali42/Fine_tuned_roberta-base-squad2'

In [52]:
# output_dir = "test-subjqa"
output_dir = "Fine_tuned_roberta-base-squad2"
repo = Repository(output_dir, clone_from=repo_name,revision="27820ad01d565335f31308687a707e86384458a4")
# repo = Repository(output_dir, clone_from=repo_name,revision="d82f6fc0add66ce43f260c5a0f0caa92c31881bc")
# repo = Repository(output_dir, clone_from=repo_name,revision="c72508ff1dcc18648bd0e4f7953f28f61172fec4")

/content/Fine_tuned_roberta-base-squad2 is already a clone of https://huggingface.co/shafieiali42/Fine_tuned_roberta-base-squad2. Make sure you pull the latest changes with `repo.git_pull()`.
Checked out 27820ad01d565335f31308687a707e86384458a4 from HEAD.



In [35]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    start_logits = []
    end_logits = []
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        # start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        # end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # start_logits = np.concatenate(start_logits)
    # end_logits = np.concatenate(end_logits)
    # print(start_logits.shape)
    # start_logits = start_logits[: len(validation_dataset)]
    # end_logits = end_logits[: len(validation_dataset)]

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    # print(start_logits.shape)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]
    # print(start_logits.shape)

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, restaurants_dataset["validation"]
    )
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/1660 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

epoch 0: {'F1': 64.63688058222061, 'EM': 63.2183908045977}
Evaluation!


...

  0%|          | 0/261 [00:00<?, ?it/s]

epoch 3: {'F1': 56.95798270639846, 'EM': 54.02298850574713}
Evaluation!


  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

epoch 4: {'F1': 56.79344736944933, 'EM': 54.02298850574713}


In [36]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

## Testing

In [96]:
# Clone the repository
!git clone https://huggingface.co/shafieiali42/Fine_tuned_roberta-base-squad2

# Navigate to the repository directory
%cd Fine_tuned_roberta-base-squad2

# Checkout the desired commit
!git checkout c72508ff1dcc18648bd0e4f7953f28f61172fec4
# !git checkout d82f6fc0add66ce43f260c5a0f0caa92c31881bc
# !git checkout b567d4aa40c537bb8f7c1b933743e1e9cbd4fc0e
# !git checkout 1f77bae21a901c110f8577f2fa2527b8b5830d19
# !git checkout 27820ad01d565335f31308687a707e86384458a4

# Load the model from the repository
# model = AutoModel.from_pretrained('<model-name>')

Cloning into 'Fine_tuned_roberta-base-squad2'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), 1.08 MiB | 1.45 MiB/s, done.
/content/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2
Note: switching to 'c72508ff1dcc18648bd0e4f7953f28f61172fec4'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch

In [97]:
my_model_checkpoint = "/content/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2/Fine_tuned_roberta-base-squad2"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# trained_checkpoint = "distilbert-base-cased-distilled-squad"
my_tokenizer = AutoTokenizer.from_pretrained(my_model_checkpoint)
my_model = AutoModelForQuestionAnswering.from_pretrained(my_model_checkpoint).to(device)
test_dataset = restaurants_dataset["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=restaurants_dataset["test"].column_names,
)
len(restaurants_dataset["test"]), len(test_dataset)


(291, 549)

In [98]:
test_set_for_model = test_dataset.remove_columns(["example_id", "offset_mapping"])
test_set_for_model.set_format("torch")
batch = {k: test_set_for_model[k].to(device) for k in test_set_for_model.column_names}

In [99]:
with torch.no_grad():
    outputs = my_model(**batch)

In [100]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [101]:
compute_metrics(start_logits, end_logits, test_dataset, restaurants_dataset["test"])

  0%|          | 0/291 [00:00<?, ?it/s]

{'F1': 70.94310805651011, 'EM': 69.41580756013745}

In [103]:
from transformers import pipeline

model_checkpoint = "shafieiali42/Fine_tuned_roberta-base-squad2"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species."""
question = "Which name is also used to describe the Amazon rainforest in English?"
question_answerer(question=question, context=context)

{'score': 0.8636225461959839,
 'start': 202,
 'end': 231,
 'answer': 'Amazonia or the Amazon Jungle'}

In [104]:
context = """
My name is Wolfgang and I live in Berlin"""
question = "Where do I live?"
question_answerer(question=question, context=context)

{'score': 0.4047725200653076, 'start': 35, 'end': 41, 'answer': 'Berlin'}

In [105]:
context = """
My name is Clara and I live in Berkeley."""
question = "What's my name?"
question_answerer(question=question, context=context)

{'score': 0.6463325023651123, 'start': 12, 'end': 17, 'answer': 'Clara'}